# OpenEO Connection to EODC Backend

In [1]:
import logging
import openeo
logging.basicConfig(level=logging.INFO)

In [2]:
# Define constants

# Connection
EODC_DRIVER_URL = "http://openeo.eodc.eu"
OUTPUT_FILE = "/tmp/openeo_eodc_output.tiff"

EODC_USER = "user1"
EODC_PWD = "Test123#"

# Data
PRODUCT_ID = "s2a_prd_msil1c"

DATE_START = "2017-01-01"
DATE_END = "2017-01-08"

IMAGE_LEFT = 652000
IMAGE_RIGHT = 672000
IMAGE_TOP = 5161000
IMAGE_BOTTOM = 5181000
IMAGE_SRS = "EPSG:32632"

# Processes
NDVI_RED = "B04"
NDVI_NIR = "B08"

In [3]:
# Connect with EODC backend
session = openeo.session(EODC_USER, endpoint=EODC_DRIVER_URL)
# Login 
token = session.auth(EODC_USER, EODC_PWD)

session

In [4]:
# Retrieve the list of available collections
collections = session.imagecollections()

list(collections)[:5]

[{'description': 'Sentinel-2 is a multispectral, high-resolution, optical imaging mission, developed by the European Space Agency (ESA) in the frame of the Copernicus program of the European Commission.',
  'product_id': 's2a_prd_msil1c',
  'source': 'ESA'}]

In [5]:
# Select collection product
s2a_prd_msil1c = session.image(PRODUCT_ID)

s2a_prd_msil1c

In [6]:
# Specifying the date range and the bounding box
timeseries = s2a_prd_msil1c.bbox_filter(left=IMAGE_LEFT, right=IMAGE_RIGHT, top=IMAGE_TOP,
                                             bottom=IMAGE_BOTTOM, srs=IMAGE_SRS)
timeseries = timeseries.date_range_filter(DATE_START, DATE_END)

timeseries

In [7]:
# Applying some operations on the data
timeseries = timeseries.ndvi(NDVI_RED, NDVI_NIR)
timeseries = timeseries.min_time()

timeseries

In [8]:
# Sending the job to the backend
job = timeseries.send_job()

job

In [ ]:
# Queue job to start the processing
status = job.queue()

In [ ]:
# Get Status of Job
status = job.status()

status

In [10]:
# Download job result
from openeo.rest.job import ClientJob
job = ClientJob(97, session)
job.download(OUTPUT_FILE)

job

<Response [200]>


In [2]:
# Showing the result
!gdalinfo -hist "/tmp/openeo_eodc_output.tiff"

Driver: GTiff/GeoTIFF
Files: /tmp/openeo_eodc_output.tiff
       /tmp/openeo_eodc_output.tiff.aux.xml
Size is 953, 622
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (10.984051868786896,46.760379340476071)
Pixel Size = (0.000281691226781,-0.000281691226781)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (  10.9840519,  46.7603793) ( 10d59' 2.59"E, 46d45'37.37"N)
Lower Left  (  10.9840519,  46.5851674) ( 10d59' 2.59"E, 46d35' 6.60"N)
Upper Right (  11.2525036,  46.7603793) ( 11d15' 9.01"E, 46d45'37.37"N)
Lower Right (  11.2525036,  46.5851674) ( 11d15' 9.01"E, 46d35' 6.60"N)
Center      (  11.1182777,  46.6727734) ( 11d 7' 5.80"E, 46d40'21.98"N)
Band 1 Block=953x2 Type=Float32, ColorInterp=